In [1]:
import pandas as pd
import glob
import os
import re

In [2]:
SCHOOL = "McGill"
keywords = ["China", "India", "Israel", "Palestine"]
data_path = f"journal_data/txt/{SCHOOL}"
output_path = f"bias_processing/data/1/{SCHOOL.lower()}_dataset.csv"
grouped_data_path = "grouped_data/csv"

In [3]:
# The output DataFrame
output_df = pd.DataFrame(columns=["date", "school", "keyword", "article"])

# Iterate through the keywords
for keyword in keywords:
    # Read the corresponding CSV file
    df = pd.read_csv(f"{grouped_data_path}/{SCHOOL}_{keyword}.csv")

    # Iterate through the dates in the DataFrame
    for date in df["date"]:
        # Format the date in the correct format for the .txt file name
        date_formatted = date.replace("-", "_")

        # Check if the .txt file exists
        txt_files = glob.glob(f"{data_path}/{date_formatted}*.txt")
        if txt_files:
            # Open the .txt file and read its content
            with open(txt_files[0], "r", encoding='utf-8', errors='ignore') as f:
                content = f.read()

            ## Noah -> Create a new if else block for each school you add to this file
            if(SCHOOL == "McGill"):
                # Split the content by the separator "The McGill Daily" repeated twice
                articles = content.split("The McGill Daily\nThe McGill Daily")

                # Iterate through the articles and check if the keyword is in the article
                for article in articles:
                    if keyword.lower() in article.lower():
                        # If the keyword is in the article, remove footer and create a new DataFrame and append it to the output DataFrame
                        cleaned_article = re.sub("Radio\nDownload file.*Powered by WordPress", "", article, flags=re.DOTALL)
                        new_row = pd.DataFrame({
                            "date": [date],
                            "school": [SCHOOL],
                            "keyword": [keyword],
                            "article": [cleaned_article.strip()]  # Remove leading/trailing white spaces
                        })
                        output_df = pd.concat([output_df, new_row], ignore_index=True)

# Add a line space between each article
output_df["article"] = output_df["article"] + "\n"

# Save the output DataFrame into a CSV file
output_df.to_csv(output_path, index=False)